In [7]:
import cv2, time, os, json
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import glob  # 폴더 내 파일 리스트 가져오는 라이브러리

# --- 사용자 설정 ---
# 1) 비디오 파일 경로
video_path = Path(r"g:\다른 컴퓨터\KHU PC\HI Lab\0. Projects\0. On going\2. Aloe inspired DEG\0. 실험자료\4. Data measurement\250830_영상 면적 분석 파일\구aloe.mp4")

# 2) 영상 한 변의 실제 길이(mm) → 픽셀-면적 환산용
side_length_mm = 10.0

In [8]:
# -*- coding: utf-8 -*-
# ============================================
# Green-영역 이분법 마스크 · 면적 계산 · 일괄 저장 스크립트
# - 프리뷰: 원본 / 오버레이 / 흑백 마스크 (3분할)
# - 키: SPACE=일시정지/재생, S=전체 저장(frames+masks+CSV), Q=종료
# - 안전 저장: cv2.imwrite 실패 시 imencode+tofile 우회
# - 추가: 마지막 민감도(슬라이더) 값을 JSON으로 저장/복원
# ============================================

base_folder   = video_path.parent / "video_analysis"
# base_folder   = Path("C:/va_out") / video_path.stem  # 경로 이슈 시 권장
frames_folder = base_folder / "frames"
masks_folder  = base_folder / "masks"
csv_path      = base_folder / f"{video_path.stem}_area.csv"
params_path   = base_folder / "last_params.json"  # ← 슬라이더 저장 파일

base_folder.mkdir(parents=True, exist_ok=True)

# ========= 유틸 =========
def odd_from_slider(x: int) -> int:
    if x <= 1: return 0
    v = x if x % 2 == 1 else x - 1
    return max(3, v)

def win_longpath(path: Path) -> str:
    s = str(path)
    if os.name == "nt":
        if len(s) >= 2 and s[1] == ":" and not s.startswith("\\\\?\\"):
            return "\\\\?\\" + s
    return s

def safe_write_image(path: Path, img) -> bool:
    try:
        path.parent.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"[ERR] make dir failed: {e}")
        return False
    ok = cv2.imwrite(win_longpath(path), img)
    if ok:
        return True
    try:
        ext = (path.suffix or ".png").lower()
        if ext not in [".png", ".jpg", ".jpeg", ".bmp", ".tiff", ".tif", ".webp"]:
            ext = ".png"
        ret, buf = cv2.imencode(ext, img)
        if ret:
            buf.tofile(win_longpath(path))
            return True
        else:
            print(f"[ERR] imencode failed for: {path}")
    except Exception as e:
        print(f"[ERR] fallback write failed for {path}: {e}")
    return False

def preprocess_and_mask(frame_bgr, G_MIN, D_R, D_B, BLUR_K, MORPH_K, use_largest):
    if BLUR_K >= 3:
        frame_bgr = cv2.medianBlur(frame_bgr, BLUR_K)
    rgb = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
    R = rgb[:, :, 0].astype(np.int16)
    G = rgb[:, :, 1].astype(np.int16)
    B = rgb[:, :, 2].astype(np.int16)
    cond = (G >= G_MIN) & (G >= R + D_R) & (G >= B + D_B)
    mask = np.zeros(G.shape, dtype=np.uint8)
    mask[cond] = 255
    if MORPH_K >= 3:
        k = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (MORPH_K, MORPH_K))
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, k, iterations=1)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, k, iterations=1)
    if use_largest:
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
        if num_labels > 1:
            areas = stats[1:, cv2.CC_STAT_AREA]
            largest = 1 + np.argmax(areas)
            mask = np.where(labels == largest, 255, 0).astype(np.uint8)
        else:
            mask[:] = 0
    return mask

def overlay_mask(frame_bgr, mask, alpha=0.45):
    color = np.zeros_like(frame_bgr); color[:] = (0, 255, 0)
    mask3 = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
    return np.where(mask3 > 0,
                    ((1 - alpha) * frame_bgr + alpha * color).astype(np.uint8),
                    frame_bgr)

def put_multiline(img, lines, org=(10, 25), scale=0.6, color=(0, 255, 255),
                  thickness=1, line_gap=30):
    x, y = org
    for line in lines:
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, (0, 0, 0), thickness + 2, cv2.LINE_AA)
        cv2.putText(img, line, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                    scale, color, thickness, cv2.LINE_AA)
        y += line_gap
    return img

# ---- 파라미터 저장/복원 ----
DEFAULT_PARAMS = {
    "G_MIN": 60, "DELTA_R": 20, "DELTA_B": 20,
    "BLUR_K_RAW": 3, "MORPH_K_RAW": 5,  # 트랙바의 '원래 값'(짝수 가능) 저장
    "LargestOnly": 1
}
def load_params():
    try:
        with open(win_longpath(params_path), "r", encoding="utf-8") as f:
            d = json.load(f)
        return {**DEFAULT_PARAMS, **d}
    except Exception:
        return DEFAULT_PARAMS.copy()

def save_params(d):
    try:
        params_path.parent.mkdir(parents=True, exist_ok=True)
        tmp = params_path.with_suffix(".json.tmp")
        with open(win_longpath(tmp), "w", encoding="utf-8") as f:
            json.dump(d, f, ensure_ascii=False, indent=2)
        os.replace(win_longpath(tmp), win_longpath(params_path))
    except Exception as e:
        print(f"[WARN] 파라미터 저장 실패: {e}")

# ========= 비디오/스케일 =========
cap = cv2.VideoCapture(str(video_path))
if not cap.isOpened():
    raise FileNotFoundError(f"영상 열기 실패: {video_path}")
ret, first = cap.read()
if not ret:
    raise RuntimeError("첫 프레임 읽기 실패")
H, W = first.shape[:2]
mm_per_px = side_length_mm / W
area_mm2_per_px = mm_per_px ** 2
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap.release()
print(f"[INFO] 해상도 {W}x{H}, 총 프레임 {total_frames}, 1px = {mm_per_px:.5f} mm, 1px² = {area_mm2_per_px:.6f} mm²")

# ========= 트랙바 =========
cv2.namedWindow("Preview", cv2.WINDOW_NORMAL)
cv2.resizeWindow("Preview", min(1500, W * 3), min(900, H * 2))

def _nothing(_): pass
cv2.createTrackbar("G_MIN",       "Preview", 0,   255, _nothing)
cv2.createTrackbar("DELTA_R",     "Preview", 0,   100, _nothing)
cv2.createTrackbar("DELTA_B",     "Preview", 0,   100, _nothing)
cv2.createTrackbar("BLUR_K",      "Preview", 0,    31, _nothing)
cv2.createTrackbar("MORPH_K",     "Preview", 0,    31, _nothing)
cv2.createTrackbar("LargestOnly", "Preview", 0,     1, _nothing)

# ---- 저장된 값으로 초기화 ----
p = load_params()
cv2.setTrackbarPos("G_MIN",       "Preview", int(p["G_MIN"]))
cv2.setTrackbarPos("DELTA_R",     "Preview", int(p["DELTA_R"]))
cv2.setTrackbarPos("DELTA_B",     "Preview", int(p["DELTA_B"]))
cv2.setTrackbarPos("BLUR_K",      "Preview", int(p["BLUR_K_RAW"]))
cv2.setTrackbarPos("MORPH_K",     "Preview", int(p["MORPH_K_RAW"]))
cv2.setTrackbarPos("LargestOnly", "Preview", int(p["LargestOnly"]))

# ========= 저장 루틴 =========
def save_all(G_MIN, D_R, D_B, BLUR_K, MORPH_K, LARGEST):
    print("[INFO] 저장 시작...")
    base_folder.mkdir(parents=True, exist_ok=True)
    frames_folder.mkdir(parents=True, exist_ok=True)
    masks_folder.mkdir(parents=True, exist_ok=True)

    cap2 = cv2.VideoCapture(str(video_path))
    if not cap2.isOpened():
        print("[ERR] 저장용 VideoCapture 열기 실패"); return

    frame_list, area_px_list, area_mm2_list = [], [], []
    j = 0; write_fail = 0
    while True:
        ret2, fr2 = cap2.read()
        if not ret2: break
        m2 = preprocess_and_mask(fr2, G_MIN, D_R, D_B, BLUR_K, MORPH_K, LARGEST)
        if LARGEST:
            num_labels, _, stats, _ = cv2.connectedComponentsWithStats(m2, connectivity=8)
            px = int(stats[1:, cv2.CC_STAT_AREA].max()) if num_labels > 1 else 0
        else:
            px = int(np.count_nonzero(m2))

        f_name = f"frame_{j:05d}.png"; m_name = f"mask_{j:05d}.png"
        ok1 = safe_write_image(frames_folder / f_name, fr2)
        ok2 = safe_write_image(masks_folder  / m_name, m2)
        if not ok1 or not ok2:
            write_fail += 1
            if not ok1: print(f"[WARN] 원본 저장 실패: {frames_folder / f_name}")
            if not ok2: print(f"[WARN] 마스크 저장 실패: {masks_folder / m_name}")

        frame_list.append(j)
        area_px_list.append(px)
        area_mm2_list.append(px * area_mm2_per_px)
        if j % 100 == 0: print(f"  processed frame {j}")
        j += 1
    cap2.release()

    try:
        df = pd.DataFrame({"frame": frame_list, "area_px": area_px_list, "area_mm2": area_mm2_list})
        csv_tmp = base_folder / f"{video_path.stem}_area_tmp.csv"
        df.to_csv(win_longpath(csv_tmp), index=False, encoding="utf-8-sig")
        os.replace(win_longpath(csv_tmp), win_longpath(csv_path))
    except Exception as e:
        print(f"[ERR] CSV 저장 실패: {e}")

    print(f"[INFO] 저장 완료:\n - frames: {frames_folder} ({j} files)\n - masks : {masks_folder} ({j} files)\n - csv   : {csv_path}\n - write_fail: {write_fail}")

# ========= 프리뷰 루프 =========
idx = 0; paused = False

def read_slider_raw():
    """트랙바 원본(짝수 포함) 값들을 그대로 읽음"""
    return {
        "G_MIN": cv2.getTrackbarPos("G_MIN", "Preview"),
        "DELTA_R": cv2.getTrackbarPos("DELTA_R", "Preview"),
        "DELTA_B": cv2.getTrackbarPos("DELTA_B", "Preview"),
        "BLUR_K_RAW": cv2.getTrackbarPos("BLUR_K", "Preview"),
        "MORPH_K_RAW": cv2.getTrackbarPos("MORPH_K", "Preview"),
        "LargestOnly": cv2.getTrackbarPos("LargestOnly", "Preview"),
    }

try:
    while True:
        idx = idx % max(1, total_frames)
        cap = cv2.VideoCapture(str(video_path))
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read(); cap.release()
        if not ret: idx += 1; continue

        # 슬라이더 값 읽기 (원본 & 변환값)
        raw = read_slider_raw()
        G_MIN   = raw["G_MIN"]
        D_R     = raw["DELTA_R"]
        D_B     = raw["DELTA_B"]
        BLUR_K  = odd_from_slider(raw["BLUR_K_RAW"])
        MORPH_K = odd_from_slider(raw["MORPH_K_RAW"])
        use_largest = bool(raw["LargestOnly"])

        mask = preprocess_and_mask(frame, G_MIN, D_R, D_B, BLUR_K, MORPH_K, use_largest)
        overlay = overlay_mask(frame, mask, alpha=0.45)
        mask_bgr = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)
        panel = np.hstack([frame, overlay, mask_bgr])

        lines = [
            f"Frame {idx}/{total_frames-1}",
            "Controls: SPACE=Pause/Play, S=Save(frames+masks+CSV), Q=Quit",
            "Tip: Preview 창이 활성화돼 있어야 키 입력이 인식됩니다.",
            f"Params: G_MIN={G_MIN}  dR={D_R}  dB={D_B}  BLUR={BLUR_K}  MORPH={MORPH_K}  LargestOnly={int(use_largest)}",
        ]
        panel = put_multiline(panel, lines, org=(10, 30))
        cv2.imshow("Preview", panel)

        key = cv2.waitKey(1 if not paused else 30) & 0xFF
        if key == ord('q'):
            # 종료 직전, 마지막 값 저장
            save_params(raw)
            break
        elif key == 32:  # SPACE
            paused = not paused
        elif key == ord('s'):
            print("[INFO] 's' 감지 → 저장을 시작합니다. (Preview 창 포커스 필요)")
            save_all(G_MIN, D_R, D_B, BLUR_K, MORPH_K, use_largest)
            # 저장 시점에도 현재 민감도 저장
            save_params(raw)

        if not paused:
            idx += 1
        else:
            time.sleep(0.02)
finally:
    cv2.destroyAllWindows()
    print("[INFO] 종료")


[INFO] 해상도 566x566, 총 프레임 49, 1px = 0.01767 mm, 1px² = 0.000312 mm²
[INFO] 종료
